In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv("df.csv")
print(df.shape)
df.head()

(1386, 12)


,ID,Name,State,Region,Applications,Admissions,Admission Rate,Enrollment,SATVR75,SATMT75,ACTEN75,ACTMT75
0,100654,Alabama A & M University,Alabama,Southeast,6560,4697,0.716006,1459,520.0,510.0,20.0,18.0
1,100663,University of Alabama at Birmingham,Alabama,Southeast,11906,10541,0.885352,2361,680.0,708.0,33.0,28.0
2,100706,University of Alabama in Huntsville,Alabama,Southeast,5781,4259,0.736724,1191,700.0,720.0,32.0,29.0
3,100724,Alabama State University,Alabama,Southeast,5974,5854,0.979913,797,534.0,516.0,22.0,19.0
4,100751,The University of Alabama,Alabama,Southeast,42421,33472,0.789043,7540,680.0,690.0,33.0,28.0


In [ ]:
df.columns

Index(['ID', 'Name', 'State', 'Region', 'Applications', 'Admissions',
       'Admission Rate', 'Enrollment', 'SATVR75', 'SATMT75', 'ACTEN75',
       'ACTMT75', 'City', 'X', 'Y', 'Lat', 'Long'],
      dtype='object')

In [ ]:
import re

colleges = pd.read_csv("Colleges_and_Universities.csv")

def standardize_text(text):
  text = text.lower()
  text = re.sub(r'\W+', '', text)
  text = " ".join(text.split(" "))
  return text

colleges["NAME"] = colleges["NAME"].apply(lambda n: standardize_text(n))

def extract_info(text):
  if "[]" in str(text): return None
  text = str(text).split()[1]
  return text

def extract_name(name):
  if "[]" in str(name): return None
  return " ".join(str(name).split()[1:]).split(" Name")[0]

cities = [extract_info(colleges[colleges["NAME"] == standardize_text(name)]["CITY"]) for name in df["Name"]]
x_pos = [extract_info(colleges[colleges["NAME"] == standardize_text(name)]["X"]) for name in df["Name"]]
y_pos = [extract_info(colleges[colleges["NAME"] == standardize_text(name)]["Y"]) for name in df["Name"]]

df["City"] = cities
df["X"] = x_pos
df["Y"] = y_pos

df = df[df["City"].notna()]
df["SAT75"] = df["SATVR75"] + df["SATMT75"]
df[["Name", "City", "X", "Y", "SAT75"]].head()

,Name,City,X,Y
0,Alabama A & M University,NORMAL,-9.636762e+06,4.134481e+06
1,University of Alabama at Birmingham,BIRMINGHAM,-9.663553e+06,3.962158e+06
2,University of Alabama in Huntsville,HUNTSVILLE,-9.644545e+06,4.126277e+06
3,Alabama State University,MONTGOMERY,-9.606391e+06,3.811228e+06
4,The University of Alabama,TUSCALOOSA,-9.745550e+06,3.923797e+06


In [ ]:
from pyproj import Transformer
transformer = Transformer.from_crs("EPSG:3857", "EPSG:4326")

lats = [transformer.transform(x, y)[0] for x, y in zip(np.array(df["X"]), np.array(df["Y"]))]
longs  = [transformer.transform(x, y)[1] for x, y in zip(np.array(df["X"]), np.array(df["Y"]))]

df["Lat"] = lats
df["Long"] = longs

df.to_csv("Better_df.csv", index=False)